<a href="https://colab.research.google.com/github/Febrian-chiperbase/gold-price-prediction-ai/blob/main/prediction_gold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install_dependencies.sh (jalankan ini di terminal Linux Debian Anda sebelum menjalankan script Python)
# sudo apt update
# sudo apt install python3 python3-pip python3-venv build-essential python3-dev -y
# python3 -m venv gold_env_v2
# source gold_env_v2/bin/activate
# pip install pandas numpy scikit-learn tensorflow keras-tuner prophet yfinance matplotlib
!pip install keras-tuner

import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt # Untuk Hyperparameter Tuning
from prophet import Prophet
import matplotlib.pyplot as plt
import logging
import os # Untuk membuat direktori

# Menonaktifkan logging INFO dari TensorFlow (opsional, untuk output yang lebih bersih)
tf.get_logger().setLevel(logging.ERROR)
# Menonaktifkan logging INFO dari cmdstanpy (digunakan oleh Prophet)
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)

# --- 0. Konfigurasi dan Pembuatan Direktori ---
IMAGES_DIR = "images_output"
MODELS_DIR = "models_output"
if not os.path.exists(IMAGES_DIR):
    os.makedirs(IMAGES_DIR)
if not os.path.exists(MODELS_DIR):
    os.makedirs(MODELS_DIR)

# --- 1. Pengumpulan Data ---
def fetch_financial_data(ticker, start_date, end_date=None, name="Data"):
    """Mengunduh data keuangan dari Yahoo Finance dan memastikan DatetimeIndex tunggal."""
    if end_date is None:
        end_date = pd.to_datetime('today').strftime('%Y-%m-%d')
    print(f"Mengunduh {name} ({ticker}) dari {start_date} hingga {end_date}...")
    try:
        data = yf.download(ticker, start=start_date, end=end_date, progress=False)
        if data.empty:
            print(f"Tidak ada data {name} yang diunduh untuk {ticker}.")
            return None

        if isinstance(data.index, pd.MultiIndex):
            print(f"Data {name} ({ticker}) memiliki MultiIndex. Mencoba menyederhanakan...")
            data_reset = data.reset_index()

            date_col_candidate = None
            if 'Date' in data_reset.columns:
                date_col_candidate = 'Date'
            elif 'level_0' in data_reset.columns:
                date_col_candidate = 'level_0'
            else:
                for col in data_reset.columns:
                    if 'date' in str(col).lower():
                        date_col_candidate = col
                        break

            if date_col_candidate:
                try:
                    data_reset[date_col_candidate] = pd.to_datetime(data_reset[date_col_candidate])
                    data = data_reset.set_index(date_col_candidate)
                    print(f"Index untuk {name} ({ticker}) disederhanakan menggunakan kolom '{date_col_candidate}'.")
                except Exception as e_setidx:
                    print(f"Gagal menyederhanakan MultiIndex untuk {name} ({ticker}) menggunakan kolom '{date_col_candidate}': {e_setidx}")
                    return None
            else:
                print(f"Tidak dapat menemukan kolom tanggal yang sesuai setelah mereset MultiIndex untuk {name} ({ticker}). Kolom: {data_reset.columns.tolist()}")
                return None
        elif 'Date' in data.columns and not isinstance(data.index, pd.DatetimeIndex):
            try:
                data['Date'] = pd.to_datetime(data['Date'])
                data = data.set_index('Date')
            except Exception as e_setcol:
                 print(f"Gagal menjadikan kolom 'Date' sebagai index untuk {name} ({ticker}): {e_setcol}")
                 return None

        if not isinstance(data.index, pd.DatetimeIndex):
            try:
                data.index = pd.to_datetime(data.index)
            except Exception as e_conv_idx:
                print(f"Gagal mengonversi index yang ada menjadi DatetimeIndex untuk {name} ({ticker}): {e_conv_idx}")
                return None
        data.index.name = 'Date'

        print(f"Data {name} ({ticker}) berhasil diunduh dan diproses. Index: {data.index.name}, Type: {type(data.index)}, nlevels: {data.index.nlevels}. Jumlah baris: {len(data)}")
        return data
    except Exception as e:
        print(f"Error umum saat mengunduh data {name} ({ticker}): {e}")
        return None


def load_and_merge_features(gold_df_input, economic_factors_config, start_date, end_date):
    """Memuat dan menggabungkan fitur harga emas dengan faktor ekonomi."""
    if gold_df_input is None or gold_df_input.empty:
        print("Error: gold_df_input awal tidak valid atau kosong.")
        return pd.DataFrame()

    gold_df = gold_df_input.copy()

    if not isinstance(gold_df.index, pd.DatetimeIndex) or gold_df.index.nlevels > 1:
        print(f"Peringatan: gold_df di load_and_merge_features masih memiliki index bermasalah. Type: {type(gold_df.index)}, nlevels: {gold_df.index.nlevels}. Mencoba perbaikan lagi...")
        if isinstance(gold_df.index, pd.MultiIndex):
            gold_df_reset = gold_df.reset_index()
            date_col_candidate = None
            if 'Date' in gold_df_reset.columns: date_col_candidate = 'Date'
            elif 'level_0' in gold_df_reset.columns: date_col_candidate = 'level_0'
            else:
                for col in gold_df_reset.columns:
                    if 'date' in str(col).lower(): date_col_candidate = col; break
            if date_col_candidate:
                try:
                    gold_df_reset[date_col_candidate] = pd.to_datetime(gold_df_reset[date_col_candidate])
                    gold_df = gold_df_reset.set_index(date_col_candidate)
                except: pass

        if not isinstance(gold_df.index, pd.DatetimeIndex):
            try: gold_df.index = pd.to_datetime(gold_df.index)
            except:
                print("Gagal memastikan DatetimeIndex tunggal untuk gold_df di load_and_merge_features.")
                return pd.DataFrame()
        gold_df.index.name = 'Date'
        if gold_df.index.nlevels > 1:
             print(f"Masih MultiIndex pada gold_df setelah perbaikan di load_and_merge: {gold_df.index.names}")
             return pd.DataFrame()

    if 'Close' not in gold_df.columns:
        print("Error: Kolom 'Close' tidak ditemukan di gold_df setelah penyesuaian index.")
        return pd.DataFrame()

    merged_df = gold_df[['Close']].copy()
    merged_df.rename(columns={'Close': 'Gold_Close'}, inplace=True)

    for factor_name, config in economic_factors_config.items():
        factor_df = fetch_financial_data(config['ticker'], start_date, end_date, name=factor_name)
        if factor_df is not None and not factor_df.empty and isinstance(factor_df.index, pd.DatetimeIndex) and factor_df.index.nlevels == 1:
            column_to_use = config.get('column', 'Close')
            if column_to_use in factor_df.columns:
                temp_series = factor_df[column_to_use]
                if not isinstance(temp_series, pd.Series):
                    if isinstance(temp_series, pd.DataFrame) and temp_series.shape[1] == 1:
                        temp_series = temp_series.iloc[:, 0]
                    else:
                        print(f"Peringatan: factor_df['{column_to_use}'] untuk {factor_name} bukan Series tunggal. Dilewati.")
                        continue

                temp_series.name = factor_name
                factor_series_to_join = temp_series

                try:
                    merged_df = merged_df.join(factor_series_to_join, how='left')
                except Exception as e_join:
                    print(f"Error saat join dengan {factor_name}: {e_join}")
                    print("Lanjutkan tanpa faktor ini.")
                    continue
            else:
                print(f"Kolom '{column_to_use}' tidak ditemukan di data {factor_name}. Faktor ini dilewati.")
        else:
            print(f"Data untuk {factor_name} tidak tersedia, kosong, atau indexnya bermasalah. Faktor ini dilewati.")

    merged_df.ffill(inplace=True)
    merged_df.bfill(inplace=True)
    merged_df.dropna(inplace=True)

    if merged_df.empty:
        print("DataFrame kosong setelah penggabungan fitur dan penghapusan NaN.")
    elif 'Gold_Close' not in merged_df.columns:
        print("Kolom 'Gold_Close' hilang setelah penggabungan. Periksa proses merge.")
        return pd.DataFrame()

    return merged_df

# --- 2. Pra-pemrosesan Data ---
def preprocess_data_multivariate_lstm(data_df, target_column='Gold_Close', sequence_length=60):
    """Pra-pemrosesan data untuk model LSTM multivariate."""
    print("Melakukan pra-pemrosesan data untuk LSTM multivariate...")
    if data_df.empty or target_column not in data_df.columns:
        print(f"DataFrame kosong atau kolom target '{target_column}' tidak ditemukan.")
        return None, None, None, None

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data_df)

    try:
        target_col_index = data_df.columns.get_loc(target_column)
    except KeyError:
        print(f"Kolom target '{target_column}' tidak ditemukan di data_df.columns.")
        return None, None, None, None

    X, y = [], []
    for i in range(sequence_length, len(scaled_data)):
        X.append(scaled_data[i-sequence_length:i, :])
        y.append(scaled_data[i, target_col_index])

    X, y = np.array(X), np.array(y)

    if X.ndim == 2 and X.shape[0] > 0 :
        X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    elif X.ndim == 1 and X.shape[0] > 0:
        X = np.reshape(X, (1, X.shape[0], data_df.shape[1] if data_df.shape[1] > 0 else 1))

    print(f"Bentuk X: {X.shape}, Bentuk y: {y.shape}")
    if X.shape[0] == 0:
        print("Tidak ada data yang dihasilkan setelah pra-pemrosesan. Periksa sequence_length atau panjang data.")
        return None, None, None, None

    return X, y, scaler, target_col_index

# --- 3. Model AI: LSTM ---
class CustomHyperModel(kt.HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape
    def build(self, hp):
        model = Sequential()
        use_bidirectional = hp.Boolean("use_bidirectional", default=False)
        hp_units_1 = hp.Int('units_1', min_value=32, max_value=256, step=32)
        add_lstm_layer_2 = hp.Boolean("add_lstm_layer_2", default=True)

        return_sequences_layer1 = add_lstm_layer_2

        if use_bidirectional:
            model.add(Bidirectional(LSTM(units=hp_units_1, return_sequences=return_sequences_layer1),
                                    input_shape=self.input_shape))
        else:
            model.add(LSTM(units=hp_units_1, return_sequences=return_sequences_layer1,
                           input_shape=self.input_shape))
        model.add(Dropout(hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)))

        if add_lstm_layer_2:
            hp_units_2 = hp.Int('units_2', min_value=32, max_value=128, step=32)
            if use_bidirectional:
                model.add(Bidirectional(LSTM(units=hp_units_2, return_sequences=False)))
            else:
                model.add(LSTM(units=hp_units_2, return_sequences=False))
            model.add(Dropout(hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)))

        model.add(Dense(units=1))
        hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                      loss='mean_squared_error')
        return model

def perform_hyperparameter_tuning(X_train, y_train, X_val, y_val, input_shape):
    """Melakukan hyperparameter tuning menggunakan KerasTuner."""
    print("Memulai Hyperparameter Tuning...")
    hypermodel = CustomHyperModel(input_shape)

    tuner = kt.Hyperband(
        hypermodel,
        objective='val_loss',
        max_epochs=30,
        factor=3,
        directory=os.path.join(MODELS_DIR, 'keras_tuner'),
        project_name='gold_lstm_tuning_v3',
        overwrite=True
    )

    stop_early = EarlyStopping(monitor='val_loss', patience=5)

    tuner.search(X_train, y_train, epochs=50, validation_data=(X_val, y_val), callbacks=[stop_early], verbose=1)

    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    print(f"""
    Hyperparameter terbaik yang ditemukan:
    Units Layer 1: {best_hps.get('units_1')}
    Dropout Layer 1: {best_hps.get('dropout_1')}
    Bidirectional: {best_hps.get('use_bidirectional')}
    Add LSTM Layer 2: {best_hps.get('add_lstm_layer_2')}
    Units Layer 2: {best_hps.get('units_2') if best_hps.get('add_lstm_layer_2') else 'N/A'}
    Dropout Layer 2: {best_hps.get('dropout_2') if best_hps.get('add_lstm_layer_2') else 'N/A'}
    Learning Rate: {best_hps.get('learning_rate')}
    """)

    model = tuner.hypermodel.build(best_hps)
    return model, best_hps

def train_best_lstm_model(model, X_train, y_train, X_val, y_val, epochs=100, batch_size=32):
    """Melatih model LSTM terbaik setelah tuning."""
    print("Melatih model LSTM terbaik...")
    stop_early = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                        validation_data=(X_val, y_val), callbacks=[stop_early], verbose=1)
    print("Pelatihan model terbaik selesai.")
    try:
        model.save(os.path.join(MODELS_DIR, "best_lstm_model.keras"))
        print(f"Model disimpan di {os.path.join(MODELS_DIR, 'best_lstm_model.keras')}")
    except Exception as e_save:
        print(f"Error saat menyimpan model: {e_save}")
    return model, history

def predict_with_lstm_multivariate(model, X_data, scaler, target_col_index, num_features, sequence_length, future_steps=30, all_data_scaled=None):
    """Membuat prediksi menggunakan model LSTM multivariate."""
    print("Membuat prediksi dengan LSTM multivariate...")

    predictions_scaled = model.predict(X_data)

    dummy_predictions = np.zeros((len(predictions_scaled), num_features))

    # predictions_scaled seharusnya (N,1) dari Dense(1).
    # Untuk assignment ke slice dummy_predictions[:, target_col_index] yang dianggap (N,1) oleh error,
    # kita pastikan predictions_scaled adalah (N,1).
    predictions_scaled_col_vector = predictions_scaled.reshape(-1, 1)

    # --- PERBAIKAN BERDASARKAN ERROR: LHS dianggap (N,1) oleh NumPy saat error broadcast ---
    # Maka, RHS juga harus (N,1). Menggunakan slice [:, start:end] untuk LHS juga memastikan LHS adalah 2D.
    dummy_predictions[:, target_col_index:target_col_index+1] = predictions_scaled_col_vector
    # --- Akhir Perbaikan ---

    predictions = scaler.inverse_transform(dummy_predictions)[:, target_col_index]

    future_predictions_list = []
    if all_data_scaled is not None and future_steps > 0 and all_data_scaled.shape[0] >= sequence_length:
        current_sequence = all_data_scaled[-sequence_length:].copy()

        for _ in range(future_steps):
            pred_input = current_sequence.reshape((1, sequence_length, num_features))
            next_pred_scaled_target = model.predict(pred_input, verbose=0)[0,0]

            new_row_scaled = current_sequence[-1, :].copy()
            new_row_scaled[target_col_index] = next_pred_scaled_target

            dummy_future_pred_row = np.zeros((1, num_features))
            dummy_future_pred_row[0, target_col_index] = float(next_pred_scaled_target)
            unscaled_future_pred_target = scaler.inverse_transform(dummy_future_pred_row)[0, target_col_index]
            future_predictions_list.append(unscaled_future_pred_target)

            current_sequence = np.append(current_sequence[1:,:], new_row_scaled.reshape(1, num_features), axis=0)

    future_predictions_array = np.array(future_predictions_list).reshape(-1,1)
    print("Prediksi masa depan LSTM multivariate selesai.")
    return predictions, future_predictions_array

# --- 4. Model Terinspirasi Bayesian: Prophet ---
def train_predict_prophet(data_df_input, target_column='Gold_Close', future_periods=30, test_size=0.2):
    """Melatih model Prophet dan membuat prediksi untuk target_column."""
    print(f"Mempersiapkan data untuk Prophet (target: {target_column})...")
    if not isinstance(data_df_input, pd.DataFrame) or data_df_input.empty:
        print("Error: data_df_input untuk Prophet bukan DataFrame yang valid atau kosong.")
        return None, None, None, None, None

    df_prophet = data_df_input.copy()

    if isinstance(df_prophet.index, pd.DatetimeIndex):
        df_prophet = df_prophet.reset_index()

    date_col_found = False
    if 'Date' in df_prophet.columns and pd.api.types.is_datetime64_any_dtype(df_prophet['Date']):
        df_prophet.rename(columns={'Date': 'ds'}, inplace=True)
        date_col_found = True
    elif 'index' in df_prophet.columns and pd.api.types.is_datetime64_any_dtype(df_prophet['index']):
         df_prophet.rename(columns={'index': 'ds'}, inplace=True)
         date_col_found = True

    if not date_col_found and 'ds' not in df_prophet.columns :
        print(f"Error: Kolom tanggal (Date/index) tidak ditemukan atau bukan datetime untuk Prophet. Kolom: {df_prophet.columns.tolist()}")
        return None, None, None, None, None

    if target_column not in df_prophet.columns:
        print(f"Error: Kolom target '{target_column}' tidak ditemukan untuk Prophet. Kolom: {df_prophet.columns.tolist()}")
        return None, None, None, None, None
    df_prophet.rename(columns={target_column: 'y'}, inplace=True)

    try:
        df_prophet['y'] = pd.to_numeric(df_prophet['y'])
        df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])
    except Exception as e:
        print(f"Error saat konversi tipe data ds/y untuk Prophet: {e}")
        return None, None, None, None, None

    df_prophet.dropna(subset=['ds', 'y'], inplace=True)
    if df_prophet.empty:
        print("Error: DataFrame untuk Prophet kosong setelah dropna ds/y.")
        return None, None, None, None, None

    df_prophet_train_ready = df_prophet[['ds', 'y']]

    train_size_prophet = int(len(df_prophet_train_ready) * (1 - test_size))
    df_train_prophet = df_prophet_train_ready.iloc[:train_size_prophet]
    df_test_prophet_actual_data_series_y = df_prophet_train_ready['y'].iloc[train_size_prophet:]

    if df_train_prophet.empty or len(df_train_prophet) < 2:
        print(f"Error: Data training untuk Prophet tidak cukup (data: {len(df_train_prophet)}) atau kosong.")
        return None, None, None, None, None

    print(f"Membangun dan melatih model Prophet dengan {len(df_train_prophet)} baris data training...")
    model_prophet = Prophet(interval_width=0.95, daily_seasonality=True)
    try:
        model_prophet.fit(df_train_prophet)
    except Exception as e:
        print(f"Error saat Prophet fit: {e}")
        return None, None, None, None, None
    print("Pelatihan Prophet selesai.")

    print("Membuat prediksi dengan Prophet...")
    periods_for_future_dates = len(df_test_prophet_actual_data_series_y) + future_periods
    if periods_for_future_dates <= 0:
        print("Tidak ada periode untuk prediksi masa depan Prophet.")
        return model_prophet, pd.DataFrame(columns=['ds', 'yhat', 'yhat_lower', 'yhat_upper']), pd.DataFrame(columns=['ds', 'yhat', 'yhat_lower', 'yhat_upper']), pd.DataFrame(columns=['ds', 'yhat', 'yhat_lower', 'yhat_upper']), pd.Series(dtype=float)

    future_dates = model_prophet.make_future_dataframe(periods=periods_for_future_dates)
    forecast = model_prophet.predict(future_dates)
    print("Prediksi Prophet selesai.")

    forecast_test_period = forecast.iloc[train_size_prophet : len(df_prophet_train_ready)]
    forecast_future_period = forecast.iloc[len(df_prophet_train_ready):]

    return model_prophet, forecast, forecast_test_period, forecast_future_period, df_test_prophet_actual_data_series_y

# --- 5. Evaluasi Model ---
def evaluate_model(y_true, y_pred, model_name="Model"):
    y_true_np = np.array(y_true).ravel()
    y_pred_np = np.array(y_pred).ravel()

    if len(y_true_np) != len(y_pred_np):
        print(f"Peringatan evaluasi {model_name}: Panjang y_true ({len(y_true_np)}) dan y_pred ({len(y_pred_np)}) tidak sama.")
        min_len = min(len(y_true_np), len(y_pred_np))
        if min_len == 0:
             print(f"Error evaluasi {model_name}: Tidak ada data yang cocok untuk dievaluasi.")
             return np.nan, np.nan
        y_true_np = y_true_np[:min_len]
        y_pred_np = y_pred_np[:min_len]

    if len(y_true_np) == 0:
        print(f"Error evaluasi {model_name}: Tidak ada data untuk dievaluasi.")
        return np.nan, np.nan

    rmse = np.sqrt(mean_squared_error(y_true_np, y_pred_np))
    mae = mean_absolute_error(y_true_np, y_pred_np)
    print(f"Evaluasi {model_name}:")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE:  {mae:.4f}")
    return rmse, mae

# --- 6. Visualisasi ---
def plot_training_history(history, model_name="LSTM"):
    if history is None or history.history is None:
        print(f"Tidak ada history pelatihan untuk model {model_name}.")
        return
    plt.figure(figsize=(10, 6))
    if 'loss' in history.history:
        plt.plot(history.history['loss'], label='Training Loss')
    if 'val_loss' in history.history:
        plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'History Pelatihan Model {model_name}')
    plt.ylabel('Loss (MSE)')
    plt.xlabel('Epoch')
    if 'loss' in history.history or 'val_loss' in history.history:
        plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(IMAGES_DIR, f'{model_name.lower()}_training_history.png'))
    print(f"Plot history pelatihan disimpan sebagai {IMAGES_DIR}/{model_name.lower()}_training_history.png")
    plt.close()

def plot_predictions_multivariate(original_target_series, train_predict_plot, test_predict_plot, future_predict_plot,
                                  model_name="LSTM", title_suffix=""):
    plt.figure(figsize=(15, 7))

    if not isinstance(original_target_series.index, pd.DatetimeIndex):
        try:
            original_target_series_index = pd.to_datetime(original_target_series.index)
        except Exception as e:
            print(f"Error converting original_target_series.index to DatetimeIndex for plotting: {e}")
            original_target_series_index = original_target_series.index
    else:
        original_target_series_index = original_target_series.index

    plt.plot(original_target_series_index, original_target_series.values, label='Harga Aktual Keseluruhan', color='blue', alpha=0.7)

    plot_index = original_target_series_index

    if train_predict_plot is not None and not np.all(np.isnan(train_predict_plot)):
        if len(train_predict_plot) == len(plot_index):
            plt.plot(plot_index, train_predict_plot,
                    label=f'Prediksi {model_name} (Train)', color='orange', linestyle='--')
        else:
            print(f"Peringatan: Panjang train_predict_plot ({len(train_predict_plot)}) tidak sama dengan plot_index ({len(plot_index)}). Plot training mungkin tidak akurat.")

    if test_predict_plot is not None and not np.all(np.isnan(test_predict_plot)):
        if len(test_predict_plot) == len(plot_index):
            plt.plot(plot_index, test_predict_plot,
                    label=f'Prediksi {model_name} (Test)', color='red', linestyle='--')
        else:
             print(f"Peringatan: Panjang test_predict_plot ({len(test_predict_plot)}) tidak sama dengan plot_index ({len(plot_index)}). Plot testing mungkin tidak akurat.")

    if future_predict_plot is not None and len(future_predict_plot) > 0:
        last_date = None
        if isinstance(plot_index, pd.DatetimeIndex) and not plot_index.empty:
            last_date = plot_index[-1]

        if last_date:
            future_dates_idx = pd.to_datetime([last_date + pd.Timedelta(days=i) for i in range(1, len(future_predict_plot) + 1)])
            plt.plot(future_dates_idx, future_predict_plot, label=f'Prediksi {model_name} (Masa Depan)', color='purple', linestyle=':')
        else:
            print("Tidak bisa membuat index tanggal untuk plot prediksi masa depan.")

    safe_title_suffix = title_suffix.replace(" ", "_").replace("(", "").replace(")", "")
    plt.title(f'Analisis Harga Emas Menggunakan {model_name}{title_suffix}')
    plt.xlabel('Tanggal')
    plt.ylabel('Harga Emas (Target)')
    plt.legend()
    plt.grid(True)
    file_name_plot = f'gold_prediction_{model_name.lower()}{safe_title_suffix}.png'
    plt.savefig(os.path.join(IMAGES_DIR, file_name_plot))
    print(f"Plot disimpan sebagai {IMAGES_DIR}/{file_name_plot}")
    plt.close()

def plot_prophet_forecast(model, forecast, original_data_prophet_ready_format, target_name="Emas (y)"):
    fig1 = model.plot(forecast)
    plt.title(f'Prediksi Harga {target_name} Menggunakan Prophet')
    plt.xlabel('Tanggal')
    plt.ylabel(f'Harga {target_name}')
    if 'ds' in original_data_prophet_ready_format.columns and 'y' in original_data_prophet_ready_format.columns:
        plt.plot(original_data_prophet_ready_format['ds'], original_data_prophet_ready_format['y'], 'k.', label='Harga Aktual')
    else:
        print("Peringatan: Kolom 'ds' atau 'y' tidak ditemukan di data original untuk plot Prophet.")
    plt.legend()
    safe_target_name = target_name.lower().replace(' (y)', '').replace('(', '_').replace(')', '').replace(' ', '_')
    file_name_detail = f'prophet_prediction_{safe_target_name}_detail.png'
    fig1.savefig(os.path.join(IMAGES_DIR, file_name_detail))
    print(f"Plot Prophet detail disimpan sebagai {IMAGES_DIR}/{file_name_detail}")
    plt.close(fig1)

    fig2 = model.plot_components(forecast)
    file_name_components = f'prophet_prediction_{safe_target_name}_components.png'
    fig2.savefig(os.path.join(IMAGES_DIR, file_name_components))
    print(f"Plot komponen Prophet disimpan sebagai {IMAGES_DIR}/{file_name_components}")
    plt.close(fig2)

# --- Main Execution ---
if __name__ == "__main__":
    # --- Konfigurasi ---
    GOLD_TICKER = 'GC=F'
    TARGET_COLUMN_NAME = 'Gold_Close'
    START_DATE = '2010-01-01'
    END_DATE = None

    ECONOMIC_FACTORS = {
        'USD_Index': {'ticker': 'DX-Y.NYB', 'column': 'Close'},
        'SP500': {'ticker': '^GSPC', 'column': 'Close'},
        'US10Y_Treasury': {'ticker': '^TNX', 'column': 'Close'},
    }

    SEQUENCE_LENGTH_LSTM = 60
    TRAIN_VAL_TEST_SPLIT_RATIO_TRAIN = 0.7
    TRAIN_VAL_TEST_SPLIT_RATIO_VAL = 0.15

    FUTURE_PREDICTION_DAYS = 30
    DO_HYPERTUNING = True
    DO_PROPHET_ANALYSIS = True

    # 1. Pengumpulan dan Penggabungan Data
    print("--- Tahap 1: Pengumpulan dan Penggabungan Data ---")
    gold_data_raw = fetch_financial_data(GOLD_TICKER, START_DATE, END_DATE, name="Emas")

    if gold_data_raw is None or gold_data_raw.empty:
        print("Tidak bisa melanjutkan tanpa data harga emas. Program berhenti.")
        exit()

    merged_full_df = load_and_merge_features(gold_data_raw, ECONOMIC_FACTORS, START_DATE, END_DATE)

    if merged_full_df.empty or TARGET_COLUMN_NAME not in merged_full_df.columns:
        print(f"Data gabungan kosong atau kolom target '{TARGET_COLUMN_NAME}' tidak ada. Menggunakan data emas saja.")
        if 'Close' in gold_data_raw.columns:
            merged_full_df = gold_data_raw[['Close']].copy()
            if not isinstance(merged_full_df.index, pd.DatetimeIndex):
                try:
                    merged_full_df.index = pd.to_datetime(merged_full_df.index)
                    merged_full_df.index.name = 'Date'
                except Exception as e_idx_fallback:
                    print(f"Gagal konversi index fallback data emas: {e_idx_fallback}. Program berhenti.")
                    exit()
            elif isinstance(merged_full_df.index, pd.DatetimeIndex):
                 merged_full_df.index.name = 'Date'

            merged_full_df.rename(columns={'Close': TARGET_COLUMN_NAME}, inplace=True)
        else:
            print("Kolom 'Close' juga tidak ditemukan di data emas mentah. Program berhenti.")
            exit()

        if merged_full_df.empty:
            print("Data emas fallback juga kosong. Program berhenti.")
            exit()

    print("\nData Gabungan (5 baris teratas):")
    print(merged_full_df.head())
    print(f"\nJumlah fitur setelah digabung (termasuk target): {merged_full_df.shape[1]}")
    print(f"Kolom data gabungan: {merged_full_df.columns.tolist()}")
    print(f"Index merged_full_df: Name='{merged_full_df.index.name}', Type='{type(merged_full_df.index)}', Levels='{merged_full_df.index.nlevels}'")


    # 2. Pemisahan Data Train, Validation, Test SEBELUM Scaling
    print("\n--- Tahap 2: Pemisahan dan Pra-pemrosesan Data untuk LSTM ---")
    total_len = len(merged_full_df)
    train_end_idx = int(total_len * TRAIN_VAL_TEST_SPLIT_RATIO_TRAIN)
    val_end_idx = train_end_idx + int(total_len * TRAIN_VAL_TEST_SPLIT_RATIO_VAL)

    train_df = merged_full_df.iloc[:train_end_idx]
    val_df = merged_full_df.iloc[train_end_idx:val_end_idx]
    test_df = merged_full_df.iloc[val_end_idx:]

    print(f"Ukuran set: Training={len(train_df)}, Validation={len(val_df)}, Test={len(test_df)}")

    if train_df.empty or val_df.empty :
        print("Data training atau validation kosong setelah pemisahan. Periksa rasio split atau jumlah data awal.")
        exit()

    X_train_lstm, y_train_lstm, scaler_lstm, target_col_idx_lstm = preprocess_data_multivariate_lstm(
        train_df, target_column=TARGET_COLUMN_NAME, sequence_length=SEQUENCE_LENGTH_LSTM
    )
    if X_train_lstm is None or X_train_lstm.size == 0:
        print("Gagal memproses data training untuk LSTM atau hasilnya kosong. Program berhenti.")
        exit()

    val_df_aligned = val_df[train_df.columns]
    scaled_val_data_full = scaler_lstm.transform(val_df_aligned)
    X_val_lstm, y_val_lstm_list = [], []
    if len(scaled_val_data_full) >= SEQUENCE_LENGTH_LSTM:
        for i in range(SEQUENCE_LENGTH_LSTM, len(scaled_val_data_full)):
            X_val_lstm.append(scaled_val_data_full[i-SEQUENCE_LENGTH_LSTM:i, :])
            y_val_lstm_list.append(scaled_val_data_full[i, target_col_idx_lstm])
        X_val_lstm = np.array(X_val_lstm)
        y_val_lstm = np.array(y_val_lstm_list)
        if X_val_lstm.ndim == 2 and X_val_lstm.size > 0: X_val_lstm = np.reshape(X_val_lstm, (X_val_lstm.shape[0], X_val_lstm.shape[1], 1))
        print(f"Data Validation LSTM: X_val: {X_val_lstm.shape}, y_val: {y_val_lstm.shape}")
    else:
        print("Data validation tidak cukup untuk membuat sequence LSTM.")
        X_val_lstm, y_val_lstm = np.array([]), np.array([])

    X_test_lstm, y_test_lstm = np.array([]), np.array([])
    if not test_df.empty:
        test_df_aligned = test_df[train_df.columns]
        scaled_test_data_full = scaler_lstm.transform(test_df_aligned)

        X_test_lstm_list_temp, y_test_lstm_list_temp = [], []
        if len(scaled_test_data_full) >= SEQUENCE_LENGTH_LSTM:
            for i in range(SEQUENCE_LENGTH_LSTM, len(scaled_test_data_full)):
                X_test_lstm_list_temp.append(scaled_test_data_full[i-SEQUENCE_LENGTH_LSTM:i, :])
                y_test_lstm_list_temp.append(scaled_test_data_full[i, target_col_idx_lstm])
            X_test_lstm = np.array(X_test_lstm_list_temp)
            y_test_lstm = np.array(y_test_lstm_list_temp)
            if X_test_lstm.ndim == 2 and X_test_lstm.size > 0: X_test_lstm = np.reshape(X_test_lstm, (X_test_lstm.shape[0], X_test_lstm.shape[1], 1))
            print(f"Data Test LSTM: X_test: {X_test_lstm.shape}, y_test: {y_test_lstm.shape}")
        else:
            print("Data test tidak cukup untuk membuat sequence LSTM.")
    else:
        print("Test DataFrame kosong.")

    # 3. Pelatihan Model LSTM
    print("\n--- Tahap 3: Pelatihan Model LSTM ---")
    best_lstm_model = None
    lstm_history = None
    num_features_lstm = X_train_lstm.shape[2] if X_train_lstm.size > 0 and X_train_lstm.ndim == 3 else (merged_full_df.shape[1] if merged_full_df.shape[1] > 0 else 1)
    input_shape_lstm = (SEQUENCE_LENGTH_LSTM, num_features_lstm)

    if X_train_lstm.size > 0 and X_val_lstm.size > 0:
        if DO_HYPERTUNING:
            best_lstm_model_tuned, best_hps = perform_hyperparameter_tuning(X_train_lstm, y_train_lstm, X_val_lstm, y_val_lstm, input_shape_lstm)
            best_lstm_model, lstm_history = train_best_lstm_model(best_lstm_model_tuned, X_train_lstm, y_train_lstm, X_val_lstm, y_val_lstm, epochs=100)
        else:
            model_path = os.path.join(MODELS_DIR, "best_lstm_model.keras")
            if os.path.exists(model_path):
                print(f"Memuat model LSTM dari {model_path}...")
                best_lstm_model = tf.keras.models.load_model(model_path)
            else:
                print("Tidak ada model tersimpan. Membuat dan melatih model LSTM default (tanpa tuning)...")
                default_hp = kt.HyperParameters()
                default_model_builder = CustomHyperModel(input_shape_lstm)
                best_lstm_model = default_model_builder.build(default_hp)
                best_lstm_model, lstm_history = train_best_lstm_model(best_lstm_model, X_train_lstm, y_train_lstm, X_val_lstm, y_val_lstm, epochs=50)

        if lstm_history:
            plot_training_history(lstm_history, model_name="LSTM_Terbaik")
    else:
        print("Data training atau validation LSTM tidak cukup, pelatihan dilewati.")

    # 4. Evaluasi Model LSTM
    print("\n--- Tahap 4: Evaluasi Model LSTM ---")
    if best_lstm_model is not None and X_test_lstm.size > 0 and y_test_lstm.size > 0 :
        pred_train_lstm_unscaled, _ = predict_with_lstm_multivariate(
            best_lstm_model, X_train_lstm, scaler_lstm, target_col_idx_lstm,
            num_features=num_features_lstm, sequence_length=SEQUENCE_LENGTH_LSTM, future_steps=0
        )

        all_data_scaled_for_future = scaler_lstm.transform(merged_full_df[train_df.columns])

        pred_test_lstm_unscaled, future_preds_lstm_unscaled = predict_with_lstm_multivariate(
            best_lstm_model, X_test_lstm, scaler_lstm, target_col_idx_lstm,
            num_features=num_features_lstm,
            sequence_length=SEQUENCE_LENGTH_LSTM,
            future_steps=FUTURE_PREDICTION_DAYS,
            all_data_scaled=all_data_scaled_for_future
        )

        dummy_y_test = np.zeros((len(y_test_lstm), num_features_lstm))

        # --- PERBAIKAN EKSPLISIT ---
        # y_test_lstm adalah 1D (N,), reshape ke (N,1) untuk assignment jika LHS dianggap demikian oleh error
        y_test_lstm_col_vector = y_test_lstm.ravel().reshape(-1, 1)

        # Mengassign (N,1) array ke slice kolom (N,1).
        # dummy_y_test[:, target_col_idx_lstm:target_col_idx_lstm+1] adalah cara untuk mendapatkan slice (N,1)
        dummy_y_test[:, target_col_idx_lstm:target_col_idx_lstm+1] = y_test_lstm_col_vector
        # --- Akhir Perbaikan ---

        y_test_lstm_unscaled = scaler_lstm.inverse_transform(dummy_y_test)[:, target_col_idx_lstm]

        evaluate_model(y_test_lstm_unscaled, pred_test_lstm_unscaled, model_name="LSTM Test")

        original_target_series = merged_full_df[TARGET_COLUMN_NAME]

        train_plot_lstm = np.full_like(original_target_series.values, np.nan, dtype=float)
        train_plot_start_idx = SEQUENCE_LENGTH_LSTM
        train_plot_end_idx = train_plot_start_idx + len(pred_train_lstm_unscaled)
        if train_plot_end_idx <= len(train_plot_lstm) and train_plot_start_idx < len(train_plot_lstm):
            train_plot_lstm[train_plot_start_idx:train_plot_end_idx] = pred_train_lstm_unscaled.ravel()

        test_plot_lstm = np.full_like(original_target_series.values, np.nan, dtype=float)
        plot_start_idx_for_test_preds = val_end_idx + SEQUENCE_LENGTH_LSTM

        if len(pred_test_lstm_unscaled) > 0:
            plot_end_idx_for_test_preds = plot_start_idx_for_test_preds + len(pred_test_lstm_unscaled)
            if plot_end_idx_for_test_preds <= len(test_plot_lstm) and plot_start_idx_for_test_preds < len(test_plot_lstm):
                test_plot_lstm[plot_start_idx_for_test_preds:plot_end_idx_for_test_preds] = pred_test_lstm_unscaled.ravel()
            else:
                 if plot_start_idx_for_test_preds < len(test_plot_lstm):
                     can_fit = len(test_plot_lstm) - plot_start_idx_for_test_preds
                     if can_fit > 0 and len(pred_test_lstm_unscaled.ravel()) >= can_fit :
                        test_plot_lstm[plot_start_idx_for_test_preds:plot_start_idx_for_test_preds+can_fit] = pred_test_lstm_unscaled.ravel()[:can_fit]
                        print(f"Peringatan: Prediksi test LSTM dipotong agar muat ({can_fit} poin).")
                     elif can_fit > 0:
                         print(f"Peringatan: Prediksi test LSTM ({len(pred_test_lstm_unscaled.ravel())}) lebih pendek dari ruang yang tersedia ({can_fit}). Tidak diplot semua.")
                     else:
                        print("Peringatan: Tidak ada ruang untuk plot prediksi test LSTM.")
                 else:
                    print(f"Peringatan: Indeks awal plot test LSTM ({plot_start_idx_for_test_preds}) di luar batas.")

        plot_predictions_multivariate(original_target_series, train_plot_lstm, test_plot_lstm,
                                      future_preds_lstm_unscaled.ravel(),
                                      model_name="LSTM", title_suffix=" dengan Faktor Ekonomi")

        print(f"\nPrediksi Harga {TARGET_COLUMN_NAME} Beberapa Hari ke Depan (LSTM):")
        last_actual_date_lstm = original_target_series.index[-1] if isinstance(original_target_series.index, pd.DatetimeIndex) else None
        if last_actual_date_lstm:
            for i, pred in enumerate(future_preds_lstm_unscaled.flatten()):
                print(f"  {(last_actual_date_lstm + pd.Timedelta(days=i+1)).strftime('%Y-%m-%d')}: {pred:.2f}")
        else:
             for i, pred in enumerate(future_preds_lstm_unscaled.flatten()): print(f"  Hari ke-{i+1}: {pred:.2f}")
    else:
        print("Model LSTM tidak dilatih atau data test tidak cukup, evaluasi dan prediksi LSTM dilewati.")

    # 5. Analisis dengan Prophet (jika diaktifkan)
    if DO_PROPHET_ANALYSIS:
        print("\n--- Tahap 5: Analisis dengan Prophet ---")
        prophet_input_df_main = merged_full_df[[TARGET_COLUMN_NAME]].copy()

        prophet_test_ratio_from_split = len(test_df) / len(merged_full_df) if len(merged_full_df) > 0 else 0.2

        if not prophet_input_df_main.empty:
            prophet_results = train_predict_prophet(
                prophet_input_df_main,
                target_column=TARGET_COLUMN_NAME,
                future_periods=FUTURE_PREDICTION_DAYS,
                test_size = prophet_test_ratio_from_split
            )

            if prophet_results and prophet_results[0] is not None:
                prophet_model, prophet_forecast, prophet_forecast_test_period, prophet_forecast_future_period, prophet_y_test_actual = prophet_results

                if prophet_y_test_actual is not None and not prophet_y_test_actual.empty and \
                   prophet_forecast_test_period is not None and not prophet_forecast_test_period.empty:
                    evaluate_model(prophet_y_test_actual.values, prophet_forecast_test_period['yhat'].values, model_name="Prophet (Test Period)")
                else:
                    print("Data aktual atau prediksi test Prophet kosong/tidak valid, evaluasi dilewati.")

                original_data_prophet_plot = prophet_input_df_main.reset_index()
                date_col_name_for_plot = 'Date'
                if date_col_name_for_plot not in original_data_prophet_plot.columns and 'index' in original_data_prophet_plot.columns:
                    date_col_name_for_plot = 'index'

                original_data_prophet_plot.rename(columns={date_col_name_for_plot: 'ds', TARGET_COLUMN_NAME: 'y'}, inplace=True)


                if prophet_forecast is not None and not prophet_forecast.empty and \
                   'ds' in original_data_prophet_plot and 'y' in original_data_prophet_plot:
                     plot_prophet_forecast(prophet_model, prophet_forecast, original_data_prophet_plot[['ds','y']], target_name=TARGET_COLUMN_NAME)
                else:
                    print("Forecast Prophet atau data original untuk plot Prophet kosong/format salah, plotting dilewati.")

                if prophet_forecast_future_period is not None and not prophet_forecast_future_period.empty:
                    print(f"\nPrediksi Harga {TARGET_COLUMN_NAME} Beberapa Hari ke Depan (Prophet):")
                    future_prophet_predictions = prophet_forecast_future_period[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
                    for index, row in future_prophet_predictions.iterrows():
                        print(f"  {row['ds'].strftime('%Y-%m-%d')}: Prediksi={row['yhat']:.2f} (Low: {row['yhat_lower']:.2f}, High: {row['yhat_upper']:.2f})")
                else:
                    print(f"Tidak ada prediksi masa depan {TARGET_COLUMN_NAME} dari Prophet.")
            else:
                print("Gagal melatih atau membuat prediksi dengan Prophet.")
        else:
            print("Data input untuk Prophet tidak valid atau kosong, analisis Prophet dilewati.")

    print("\nAnalisis selesai.")
    print(f"Plot disimpan di direktori: {IMAGES_DIR}")
    print(f"Model LSTM (jika dilatih/disimpan) ada di direktori: {MODELS_DIR}")

Trial 90 Complete [00h 00m 46s]
val_loss: 0.00473076431080699

Best val_loss So Far: 0.0002859888190869242
Total elapsed time: 02h 50m 34s

    Hyperparameter terbaik yang ditemukan:
    Units Layer 1: 32
    Dropout Layer 1: 0.1
    Bidirectional: False
    Add LSTM Layer 2: False
    Units Layer 2: N/A
    Dropout Layer 2: N/A
    Learning Rate: 0.01
    
Melatih model LSTM terbaik...
Epoch 1/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - loss: 0.0163 - val_loss: 0.0037
Epoch 2/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.0013 - val_loss: 5.4770e-04
Epoch 3/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - loss: 9.6273e-04 - val_loss: 6.2798e-04
Epoch 4/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 8.4403e-04 - val_loss: 6.7543e-04
Epoch 5/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 8.1141e-04 - val_loss: 5.3183e-04
Epoch 6/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - loss: 7.3804e-04 - val_loss: 4.0755e-04
Epoch 7/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss

TypeError: unsupported operand type(s) for +: 'slice' and 'int'